# Spark DataFrame Basics

Spark DataFrames are the workhouse and main way of working with Spark and Python post Spark 2.0. DataFrames act as powerful versions of tables, with rows and columns, easily handling large datasets. The shift to DataFrames provides many advantages:
* A much simpler syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs
    
If you've used R or even the pandas library with Python you are probably already familiar with the concept of DataFrames. Spark DataFrame expand on a lot of these concepts, allowing you to transfer that knowledge easily by understanding the simple syntax of Spark DataFrames. Remember that the main advantage to using Spark DataFrames vs those other programs is that Spark can handle data across many RDDs, huge data sets that would never fit on a single computer. That comes at a slight cost of some "peculiar" syntax choices, but after this course you will feel very comfortable with all those topics!

Let's get started!

## Creating a DataFrame

First we need to start a SparkSession:

In [1]:
from pyspark.sql import SparkSession

Then start the SparkSession

In [2]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()


You will first need to get the data from a file (or connect to a large distributed file like HDFS, we'll talk about this later once we move to larger datasets on AWS EC2).

In [3]:
# We'll discuss how to read other options later.
# This dataset is from Spark's examples

# Might be a little slow locally
df = spark.read.json('people.json')

#### Showing the data

In [4]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
# Remember to use camelScript in pyspark
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df.columns

['age', 'name']

In [8]:
# df.describe() - MUST ADD .show() TO OBTAIN DISTRIBUTION
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



Some data types make it easier to infer schema (like tabular formats such as csv which we will show later). 

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [9]:
# CHECK SCHEMA ON INITIAL DF READ & USE pyspark.sql.types TO FORMAT IF NEEDED
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Next we need to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [10]:
# 1. Define data type in schema
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [11]:
# 2. Apply schema to data fields
final_struc = StructType(fields=data_schema)

In [12]:
# 3. Reference schema when creating df
df = spark.read.json('people.json', schema=final_struc)

In [13]:
# WHAT CHANGED? age now int rather than long. Could save memory and time?
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Grabbing the data

In [14]:
# df syntax returns an object
df['age']

Column<b'age'>

In [15]:
type(df['age'])

pyspark.sql.column.Column

In [16]:
# .select method returns a df
df.select('age')

DataFrame[age: int]

In [19]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [20]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



Multiple Columns:

In [21]:
df.select(['age','name'])

DataFrame[age: int, name: string]

In [22]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [23]:
# Adding a new column with a simple copy
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [24]:
# MUST ASSIGN NEW NAME TO SAVE NEW COLUMN
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [26]:
# CORRECT SYNTAX TO CREATE A NEW COL IN A DF?
df['newage'] = df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



TypeError: 'DataFrame' object does not support item assignment

In [ ]:
# FROM STACK OVERFLOW

'''
from pyspark.sql.functions import col, expr, when

new_column_1 = expr(
    """IF(fruit1 IS NULL OR fruit2 IS NULL, 3, IF(fruit1 = fruit2, 1, 0))"""
)

new_column_2 = when(
    col("fruit1").isNull() | col("fruit2").isNull(), 3
).when(col("fruit1") == col("fruit2"), 1).otherwise(0)

(df
    .withColumn("new_column_1", new_column_1)
    .withColumn("new_column_2", new_column_2)
    .withColumn("new_column_3", new_column_3))
'''

In [27]:
# Simple Rename
df.withColumnRenamed('age','supernewage').show()

+-----------+-------+
|supernewage|   name|
+-----------+-------+
|       null|Michael|
|         30|   Andy|
|         19| Justin|
+-----------+-------+



More complicated operations to create new columns

In [28]:
df.withColumn('doubleage',df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [29]:
df.withColumn('add_one_age',df['age']+1).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
+----+-------+-----------+



In [30]:
df.withColumn('half_age',df['age']/2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+



In [31]:
df.withColumn('half_age',df['age']/2)

DataFrame[age: int, name: string, half_age: double]

We'll discuss much more complicated operations later on!

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [32]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [35]:
# NOW USE spark.sql (NOT pyspark.sql) to run SQL queries
sql_results = spark.sql("SELECT * FROM people")

In [36]:
sql_results

DataFrame[age: int, name: string]

In [37]:
sql_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [38]:
spark.sql("SELECT * FROM people WHERE age=30").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



We won't really be focusing on using the SQL syntax for this course in general, but keep in mind it is always there for you to get you out of bind quickly with your SQL skills!

Alright that is all we need to know for now!

In [40]:
# Try other SQL queries

spark.sql("SELECT * FROM people WHERE name in('Andy')").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [51]:
spark.sql("SELECT * FROM people WHERE name LIKE '%A%')").show()

ParseException: "\nextraneous input ')' expecting <EOF>(line 1, pos 42)\n\n== SQL ==\nSELECT * FROM people WHERE name LIKE '%A%')\n------------------------------------------^^^\n"

In [45]:
spark.sql("SELECT * FROM people WHERE name not in('Andy')").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  19| Justin|
+----+-------+



In [46]:
spark.sql("SELECT * FROM people WHERE age between 18 and 24").show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+



In [47]:
spark.sql("SELECT * FROM people WHERE age > 25").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [49]:
spark.sql("SELECT * FROM people WHERE age > 25 or age <= 20").show()

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+



In [55]:
spark.sql("SELECT * FROM people WHERE age != 19").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

